In [1]:
import json
from pathlib import Path
import os
import glob
from dotenv import load_dotenv
import pandas as pd

In [2]:
load_dotenv()

True

In [3]:
kaggle_dataset_path = Path(os.getenv("KAGGLE_DATASET_PATH"))

In [4]:
dirname = kaggle_dataset_path/'subset/rephrased_subset' 

In [5]:
fnames = glob.glob(str(dirname/'*.json'))

In [6]:
csv_dir = kaggle_dataset_path/"subset/triplet_csvs"
csv_dir.mkdir(exist_ok=True)

In [7]:
from trextractor import KGTriplets 
kgtriplets = KGTriplets(use_minio=False)

/Users/rbiswas/soft/miniforge3/envs/trextractor/lib/python3.10/site-packages/trextractor-0.0.1-py3.10.egg/trextractor/prompts


In [8]:
def write_kgtrip(fname, csv_dir, kgtrip=kgtriplets, model="gpt-4o"):
    with open(fname, 'r') as f:
        data = json.load(f)


    text = data['rephrased_text']
    uuid = data['uuid']
    response = kgtriplets.get_knowledge_graph_representation(text, model=model,
                                                          use_instructor=True)
    input_df, output_df = KGTriplets.wrap(response)
    output_df['uuid'] = uuid
    output_df['text'] = text
    # print(fname)
    outname = csv_dir/f"{Path(fname).name.split('.json')[0] +'.csv'}"
    output_df.to_csv(outname, index=False)  
    return output_df



In [9]:
print(f"There are {len(fnames)} files to process \n" )
bads = []
for i, fname in enumerate(fnames):
    try:
        out = write_kgtrip(fname, csv_dir, kgtriplets)
    except Exception as e:
        print(fname)
        bads.append((fname, e))
    
    if i % 5 == 0:
        print(f"{i=} and there are {len(bads)=} failures\n")

There are 100 files to process 

i=0 and there are len(bads)=0 failures

/Users/rbiswas/doc/sebx/data/kaggle_financial_data/subset/rephrased_subset/news_0023363.json
i=5 and there are len(bads)=1 failures

i=10 and there are len(bads)=1 failures

/Users/rbiswas/doc/sebx/data/kaggle_financial_data/subset/rephrased_subset/news_0004913.json
i=15 and there are len(bads)=2 failures

/Users/rbiswas/doc/sebx/data/kaggle_financial_data/subset/rephrased_subset/news_0018930.json
i=20 and there are len(bads)=3 failures

/Users/rbiswas/doc/sebx/data/kaggle_financial_data/subset/rephrased_subset/news_0011212.json
/Users/rbiswas/doc/sebx/data/kaggle_financial_data/subset/rephrased_subset/news_0008463.json
i=25 and there are len(bads)=5 failures

i=30 and there are len(bads)=5 failures

i=35 and there are len(bads)=5 failures

/Users/rbiswas/doc/sebx/data/kaggle_financial_data/subset/rephrased_subset/news_0000053.json
i=40 and there are len(bads)=6 failures

i=45 and there are len(bads)=6 failures

i

In [10]:
with open('bad_files.txt', 'w') as f:
    for bad in bads:
        f.write(f"{bad[0]},{bad[1]}\n")